In [1]:
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pandas as pd
import numpy as np
import re
import pickle
from collections import namedtuple

%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
DIR = Path('./1511337619/')
FILES = list(DIR.glob('*train*.json'))
len(FILES)

120

In [3]:
# FILES = FILES[:10]

In [4]:
pat = re.compile(r's_([0-9_]+)_a_(\w+)_f_(\d+)_bs_(\d+)_lr_([0-9._]+)_train')

model = namedtuple('model', ['shape', 'activation', 'features', 'learning_rate'])

def foo(s):
    parts = list(pat.findall(s)[0])
    del parts[3]  # remove batch size
    return model(*parts)

s = FILES[0].stem
print(foo(s))
s

model(shape='24_16_12_8', activation='sigmoid', features='10', learning_rate='0.1_0.01')


's_24_16_12_8_a_sigmoid_f_10_bs_10_lr_0.1_0.01_train'

In [5]:
testing = {}
training = {}
files = []

for file in FILES:
    s = foo(file.stem)
    files.append(s)
    with file.open() as f:
        j = json.load(f)
    testing[s] = pd.DataFrame(j['testing']).set_index('epoch')
    training[s] = pd.DataFrame(j['training']).set_index('step')

data = {'files': files, 'testing': testing, 'training': training}
files

[model(shape='24_16_12_8', activation='sigmoid', features='10', learning_rate='0.1_0.01'),
 model(shape='16_12_8', activation='tanh', features='30', learning_rate='0.2_0.2'),
 model(shape='8', activation='tanh', features='20', learning_rate='0.2_0.001'),
 model(shape='24_16_12_8', activation='tanh', features='10', learning_rate='0.1_0.01'),
 model(shape='16_8', activation='sigmoid', features='30', learning_rate='0.2_0.001'),
 model(shape='16_8', activation='sigmoid', features='20', learning_rate='0.1_0.1'),
 model(shape='24_16_12_8', activation='sigmoid', features='20', learning_rate='0.1_0.1'),
 model(shape='16_8', activation='sigmoid', features='10', learning_rate='0.2_0.2'),
 model(shape='16_12_8', activation='sigmoid', features='30', learning_rate='0.1_0.01'),
 model(shape='24_16_12_8', activation='sigmoid', features='20', learning_rate='0.2_0.01'),
 model(shape='8', activation='sigmoid', features='20', learning_rate='0.2_0.01'),
 model(shape='16_8', activation='sigmoid', features=

In [6]:
with open('./data/tests.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
testing[list(testing.keys())[0]]

In [ ]:
training[list(training.keys())[0]]

In [ ]:
for k, v in data.items():
    acc = v['test_accuracy']
    if acc.max() < .88:
        continue
    print(acc.max())
    acc.plot(label=k)
plt.legend()
plt.show()

In [ ]:
data[list(data.keys())[0]].describe()